# TILT - Teemo Induced Loss of Tranquility
A research insentive to study main factors in inducing tilt.


# Imports

In [1]:
from collections import defaultdict
from pathlib2 import Path
from riotwatcher import LolWatcher, ApiError


# import arrow
import datetime
import time
import json
import numpy as np
import pandas as pd

# Functions

In [9]:
  
def api_key(api_key_loc):
    """
    Read in the development API key from a file and checks if it is viable.
    
    Args:
        credentials (str): Name of json file containing the credentials.
    Returns:
        api_key (str): The API key.
    """

    while True:
        with open(api_key_loc, "r") as json_data:
            creds = json.load(json_data)
            api_key = creds["dev_api_key"]
            lol_watcher = LolWatcher(creds["dev_api_key"])
            try:
                # Validate API key by using it to check server status
                lol_watcher.lol_status.shard_data("euw1")
                # Break if key is functional
                break
            except ApiError as error:
                # If the current API key does not work input new one
                if error.response.status_code == 403:
                    new_api_key = input("API key is incorrect, enter correct key here.")
                    creds["dev_api_key"] = new_api_key
                    # Replace the old API key
                    with open(api_key_loc, "w") as json_data:
                        json.dump(creds, json_data)
    return api_key

In [20]:
def convert_time(utc_time):
    """
    Converts UTC to GMT time.

    Args:
        time (int): Time in milliseconds following the Coordinated Universal Time (UTC) format.
    Returns:
        hr_time (str): Time in Greenwich Mean Time format (Year-Month-Day Hour:Minute:Second). 
    """
    
    gmt_time = str(datetime.datetime.fromtimestamp(time / 1000))
    return(gmt_time)

In [27]:
def kda(kill, death, assist):
    #somehow does not work
    """
    Calculate kill assist to death ratio. 
    If a player would have 0 deaths, a value of 1 will be used to avoid a KDA of infinity  
    
    Args:
        kill (str): Time in milliseconds following the Coordinated Universal Time (UTC) format.
        death (str):
        assists (str)
    Returns:
        hr_time (str): Time in Greenwich Mean Time format (Year-Month-Day Hour:Minute:Second). 
    """
    Input is string

    """
    score = round((kill+assist) / max(1, death), 3)

    return score

In [30]:
print(kda(5,1,5))

10.0


# Main

In [14]:
# Folder locations

# Project folders
project_folder = Path.cwd().parent

# Raw data storage
data_folder = project_folder / "data"

# Out folder
out_folder = project_folder / "out"

In [15]:
# Set API key

# API key location
api_key_loc = data_folder / "dev_api_key.json"

# Set API key
lol_watcher = LolWatcher(api_key(api_key_loc))

In [26]:
# Constants

# Minimum durtion of champion select according to https://leagueoflegends.fandom.com/wiki/Draft_Pick 7-02-2021
min_lobby_time = 239*1000 


In [19]:
# Set user request parameters

# Set region
region = "euw1"

# Set summonder name 
protagonist = "alnas" # not case sensitive

### User info

In [18]:
# retrieve info using region and summoner name
user = lol_watcher.summoner.by_name(region, protagonist)
print(user)

{'id': 'C6R-3UT4OoZ5pd5v3cbLAO6CQeKGzESmAKsI_ZKcAOioKf0', 'accountId': 'Kk8PvJLYQzu-GbSacqZqL3yPXjn_BvEpzc8aT86BPqX0gg', 'puuid': 'pmzyZf0APVn-_KRaTzgA3inKNh5_CSKYuWQX3c0JxUGs1f9kxjBmstXG2W-bY4Nz-Z4gP3gSEmlRDA', 'name': 'Aderynn', 'profileIconId': 4491, 'revisionDate': 1644051914000, 'summonerLevel': 67}


In [29]:
# Check rank stats using user id
ranked_stats = lol_watcher.league.by_summoner(region, user["id"])
print(ranked_stats)

[{'leagueId': 'f45c0593-8eab-41ca-a45e-987f2cff8cca', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'GOLD', 'rank': 'IV', 'summonerId': 'L2OqMzNqsCelFfcFlPx5iQWRB7dKLDdf9xGgO_W1dNau3EI', 'summonerName': 'Alnas', 'leaguePoints': 50, 'wins': 16, 'losses': 18, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


### Match info

In [30]:
# set match request parameters

# Set queue
queue = 420

# Set season ????

In [31]:
# Retrieve match id from a single match, ranked queue based on time
# (max is last 100 matches https://developer.riotgames.com/apis#match-v4) but time range is 1 week 


matches = lol_watcher.match.matchlist_by_account(region=region,
                                                 encrypted_account_id=user["accountId"],
                                                 queue=queue,
                                                 begin_index=0,
                                                 end_index=100)
#TODO: make this work with time

                                                 
# print(matches.keys())
# print(matches["matches"][0])
# print(matches["startIndex"])
# print(matches["endIndex"])
# print(matches["totalGames"]) 


AttributeError: 'MatchApiV4' object has no attribute 'matchlist_by_puuid'

In [24]:
# Create a dictionary to store all the results
match_info = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

# Get match details of last 10 matches
for match in matches["matches"][:1]:
        # Collect information pertinent to the match itself
        game_id = match["gameId"]
        queue_type = match["queue"]
        match_detail = lol_watcher.match.by_id(region, game_id)
        match_start = match["timestamp"]
        match_duration = match_detail["gameDuration"]*1000
        lobby_end = match_detail["gameCreation"]

        # Store filtered information pertinent to the match itself
        match_info[game_id]["game_info"]["game_type"] = queue_type
        match_info[game_id]["game_info"]["begin_time"] = lobby_end - min_lobby_time
        match_info[game_id]["game_info"]["end_time"] = match_start + match_duration

        # Store filtered information pertinent to participants
        protagonist_blue = None
        # Collect participant ID and summoner name
        for participant in match_detail["participantIdentities"]:
                part_id = participant["participantId"]
                summoner_name = participant["player"]["summonerName"]
                # Storing summoner name using participant ID as key
                match_info[game_id]["player_data"][part_id]["participant_id"] = part_id 
                match_info[game_id]["player_data"][part_id]["name"] = summoner_name
                if summoner_name.lower() == protagonist.lower():
                        protagonist_blue = True if part_id <=5 else False

        # Collect kills, deaths, assists and win
        for participant in match_detail["participants"]:
                part_id = participant["stats"]["participantId"]
                kills = participant["stats"]["kills"] 
                deaths = participant["stats"]["deaths"]
                assists = participant["stats"]["assists"]
                kda = round((kills+assists)/ max(1, deaths), 3)
                win = participant["stats"]["win"]

                # Store kills, deaths, assists, kda and win
                match_info[game_id]["player_data"][part_id]["kills"] = kills
                match_info[game_id]["player_data"][part_id]["deaths"] = deaths
                match_info[game_id]["player_data"][part_id]["assists"] = assists
                match_info[game_id]["player_data"][part_id]["kda"] = kda
                match_info[game_id]["player_data"][part_id]["win"] = win

                # Assign participants as ally or enemy of the protagonist                   
                match_info[game_id]["player_data"][part_id]["allied"] = bool(part_id <=5) == protagonist_blue

print("done", datetime.datetime.now())
# TODO: print how long it took

NameError: name 'matches' is not defined

In [23]:
# Output for for easy reading
for i in match_info.keys():
        print("this is match", i)
        print(match_info[i].keys())
        print(match_info[i]["game_info"].keys())
        print(convert_time(match_info[i]["game_info"]["begin_time"]))
        print(convert_time(match_info[i]["game_info"]["end_time"]))
        print("")

        print("match info")
        for y in match_info[i]["game_info"]:
                print(match_info[i]["game_info"][y])

        print("")
        print("player info")
        for player in match_info[i]["player_data"]:
                print(match_info[i]["player_data"][player])

        print("\n------------------------\n")

In [ ]:
# todo: write a function that measures how long a block took to run
# datetime.now()

In [71]:
with open(data_folder / "patches.json") as f:
    data = json.load(f)

patch = data["patches"][-1]  # most recent
utc_timestamp = patch["start"]

north_america_timestamp = utc_timestamp + data["shifts"][region.upper()]
print(north_america_timestamp)
dt = arrow.get(north_america_timestamp)
print(dt.to("US/Pacific"))

1607493600
2020-12-08T22:00:00-08:00


In [ ]:

# participants = []
# for row in match_detail['participants']:
#     participants_row = {}
#     participants_row['champion'] = row['championId']
#     # participants_row['spell1'] = row['spell1Id']
#     # participants_row['spell2'] = row['spell2Id']
#     participants_row['win'] = row['stats']['win']
#     participants_row['kills'] = row['stats']['kills']
#     participants_row['deaths'] = row['stats']['deaths']
#     participants_row['assists'] = row['stats']['assists']
#     participants_row['KDA'] = kda(row['stats']['kills'], row['stats']['deaths'], row['stats']['assists'])
#     participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
#     participants_row['goldEarned'] = row['stats']['goldEarned']
#     participants_row['champLevel'] = row['stats']['champLevel']
#     participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
#     # participants_row['item0'] = row['stats']['item0']
#     # participants_row['item1'] = row['stats']['item1']
#     participants.append(participants_row)
# df = pd.DataFrame(participants) # this step is still important 07-2
# # df

In [44]:

thingy = np.arange(220,551,110)

print(thingy)

[220 330 440 550]


### Patch an season info

In [62]:
# example of how to Read time
# 2018-01-24T03:00:00-08:00  (for patch 8.2) --> corresponds to 3am on January 24th, 2018 in Pacific Time

with open(data_folder / "patches.json") as f:
    data = json.load(f)

patch = data["patches"][-1]  # most recent
utc_timestamp = patch["start"]
north_america_timestamp = utc_timestamp + data["shifts"]["NA1"]
print(north_america_timestamp)
dt = arrow.get(north_america_timestamp)
print(dt.to("US/Pacific"))



1607515200
2020-12-09T04:00:00-08:00


In [ ]:
# example end script for wayyy later

try:
    response = lol_watcher.summoner.by_name(region, summoner_name)
    print(response)
except ApiError as error:
    if error.response.status_code == 429:
        print('Request limit reached retry in {} seconds.'.format(err.headers['Retry-After']))
    elif error.response.status_code == 404: 
        print("Found no match")